# **Launch Sites Locations Analysis with Folium**

This lab contains the following tasks:
-  Mark all launch sites on a map
-  Mark the success/failed launches for each site on the map
-  Calculate the distances between a launch site to its proximities

In [18]:
# Import the necessary libraries
import pandas as pd # For data manipulation and analysis
import folium       # For creating maps
import wget         # For downloading files

from folium.plugins  import MarkerCluster       # For clustering the markers
from folium.plugins  import MousePosition       # For displaying the mouse position
from folium.features import DivIcon             # For adding text to the map
from folium.plugins import MousePosition        # For displaying the mouse position
from math import sin, cos, sqrt, atan2, radians # For calculating the distance between two points

## Geospatial Visualization

In [19]:
# Read the SpaceX dataset into a Pandas data frame 
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df = pd.read_csv(spacex_csv_file)

# Select relevant sub-columns.
spacex_df       = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Display the data frame
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [20]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, 
                       radius =  1000, 
                       color  = '#0000CD', 
                       fill   =  True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,

    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#0000CD;"><b>%s</b></div>' % 'NASA JSC',
        )
    )

# Add the blue circle and marker to the site map
site_map.add_child(circle)
site_map.add_child(marker)

In [21]:
# Create and add folium Circle and folium Marker for each launch site on the site map

# Initial the map
site_map = folium.Map(location=[33.57786, -101.85517], zoom_start=4)

# Add a Circle object based on its coordinate values. In addition, add Launch site name as a popup label
launch_sites = [
    {"location": [28.562302, -  80.577356], "popup": "Space X Launch Site CCAFS LC-40",  "marker_text": "CCAFS LC-40"},
    {"location": [28.563197, -  80.576820], "popup": "Space X Launch Site CCAFC SLC-40", "marker_text": "CCAFS SLC-40"},
    {"location": [28.573255, -  80.646895], "popup": "Space X Launch Site KSC LC-39A",   "marker_text": "KSC LC-39A"},
    {"location": [34.632834, - 120.610746], "popup": "Space X Launch Site VAFB SLC-4E",  "marker_text": "VAFB SLC-4E"}
]

# for loop to add each launch site to the map
for site in launch_sites:
    folium.Circle(
        location =  site["location"],
        radius   =  1000,
        color    = '#2534BD',
        fill     =  True
    ).add_child(folium.Popup(site["popup"])).add_to(site_map)

    # Add a folium.Marker to the map
    folium.map.Marker(
        location    =  site["location"],
        icon        =  DivIcon(
        icon_size   = (20,20),
        icon_anchor = (0,0),
        html        = '<div style="font-size: 12; color:#2534BD;"><b>%s</b></div>' % site["marker_text"],)
        ).add_to(site_map)
    

# Display the map
site_map

In [22]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

In [23]:
# Create a Marker clusters object to simplify the map
marker_cluster = MarkerCluster()

# Create a new column in 'launch_sites' data frame called 'marker_color' to store the marker colors based on the 'class' value
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Display the last 10 rows of the data frame to verify the changes
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [24]:
# Mark the success/failed launches for each site on the map

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Create a marker object for each launch site 
for index, record in spacex_df.iterrows():
    
    # Create and add a Marker cluster to the site map
    marker   =  folium.Marker(
    location = [record['Lat'], record['Long']],
    popup    =  record['Launch Site'],
    icon     =  folium.Icon(color      = 'white', 
                            icon_color =  record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

In [25]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position    =  MousePosition(
    position      = 'topright',
    separator     = 'Long: ',
    empty_string  = 'NaN',
    lng_first     =  False,
    num_digits    =  20,
    prefix        = 'Lat:',
    lat_formatter =  formatter,
    lng_formatter =  formatter,     
)

# Add the Mouse Position to the map
site_map.add_child(mouse_position)

# Display the map
site_map

In [26]:
# Function to calculate the distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate the distance
    distance = R * c
    return distance

In [27]:
# Calculate the distance between the coastline point and the launch site.

# Select a record in 'spacex_df' for a specific launch site
row_index = 50

# Select the launch site lat and log
launch_site_lat =  spacex_df.loc[row_index, 'Lat']
launch_site_lon =  spacex_df.loc[row_index, 'Long']
coast_line_lat  =  28.56405
coast_line_lon  = -80.56809

# Calculate the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coast_line_lat, coast_line_lon)

# Display the distance
distance_coastline

0.8581105059079531

In [28]:
# Create and add a folium.Marker on your selected closest coastline point on the map
distance_marker    =  folium.Marker(
          location = [coast_line_lat, coast_line_lon],
              icon =  DivIcon(
       icon_size   = (20,20),
       icon_anchor = (0,0),
              html = '<div style="font-size: 12; color:#2534BD;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   ) 

# Create a folium.PolyLine object using the coastline coordinates and launch site coordinate
launch_site_coordinates = [launch_site_lat, launch_site_lon]
coast_line_coordinates  = [coast_line_lat, coast_line_lon]

# Create a line object
lines = folium.PolyLine(locations = [[launch_site_coordinates, coast_line_coordinates]], weight=1)

# Add the distance marker and line to the site map
site_map.add_child(lines)

In [29]:
# Create a folium.PolyLine from the launch site to the city of Orlando, Florida

# orlando city coordinate 
orlando_city_lat = 28.53834
orlando_city_lon = -81.37924

# distance between orlando city and launch site
orlando_distance = calculate_distance(launch_site_lat, launch_site_lon, orlando_city_lat, orlando_city_lon)

# distance marker
distance_marker =  folium.Marker(
       location = [orlando_city_lat, orlando_city_lon],
           icon =  DivIcon(
      icon_size = (20,20),
    icon_anchor = (0,0),
           html = '<div style="font-size: 12; color:#2534BD;"><b>%s</b></div>' % "{:10.2f} KM".format(orlando_distance),
       )
   ) 

# add distance marker to site_map
site_map.add_child(distance_marker)

# Create a folium.PolyLine object using the coordinates of the launch site and the closest city
launch_site_coordinates  = [launch_site_lat, launch_site_lon]
orlando_city_coordinates = [orlando_city_lat, orlando_city_lon]

# Create a folium.PolyLine object using the coastline coordinates and launch site coordinate
lines = folium.PolyLine(
    locations = [[launch_site_coordinates, orlando_city_coordinates]], 
        weight =  1,
         color = 'blue' 
)

# add line to site_map
site_map.add_child(lines)

This Jupyter Notebook was Completed By [Jonathan Scott](http://www.linkedin.com/in/jonathan-scott-140709317) as Part of the IBM Data Science Certification.

## Author

[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)

## Other Contributors

Joseph Santarcangelo

## Change Log

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|

Copyright © 2021 IBM Corporation. All rights reserved.